This is a how to guide for importing data from .csv file to Django database (SQLite). You will see example data classes. Please note that those classes are for demonstration only. You are free to create classes in the way you see fit the best. 

The end goal is to create a JSON data that will be used to import data by Django built-in command. The format we want is structured as below. (more detail check -> https://docs.djangoproject.com/en/5.1/howto/initial-data/)

```JSON
[
  {
    "model": "myapp.classname",
    "pk": 1,
    "fields": {
      "attribute_name1": "Value",
      "attribute_name2": "Value"
    }
  },
  {
    "model": "myapp.field",
    "pk": 2,
    "fields": {
      "field_id": "32",
      "field_name": "Corner"
    }
  }
]

```

In Python, you can assume that JSON is Dictionary-like data type. So, the overall step is .csv -> Pandas dataframe -> dictionary ->JSON. 

Let's start with importing necessary modules. In this case, we only need 2 modules. Then load data from .csv to Pandas dataframe (we did this before in lab 2).

In [1]:
# import python's module
import pandas as pd
import json

In [2]:
# load data
log_df = pd.read_csv("operation-log.csv")  
log_df
# _df stands for data frame. It is a common suffix to indicate variable type. 

,date,operator,location,operation,note,Power Unit,Seeds planted,Seeding Rate (seeds/ac),Fertilizers applied,Fertilizer Rate (lb/ac)
0,4/24/2022,Evan,200,Spread/Spray,2-4D round up burn down,Hagie STS12,NaN,NaN,NaN,NaN
1,4/27/2022,Bryan,6,Spread/Spray,"20oz ru, 11 24d",Hagie STS12,NaN,NaN,NaN,NaN
2,4/27/2022,Bryan,5,Spread/Spray,20 oz ru/ 11 oz 24d,Hagie STS12,NaN,NaN,NaN,NaN
3,4/27/2022,Bryan,Cotton,Spread/Spray,20 oz ru. 11 oz 24d,Hagie STS12,NaN,NaN,NaN,NaN
4,4/27/2022,Bryan,101,Spread/Spray,20 oz ru 11 oz 24d,Hagie STS12,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
428,11/17/2022,Ceres,105,Spread/Spray,VRF MAP: 153 lbs/ac\r\nVRF Potash: 133 lbs/ac,NaN,NaN,NaN,"MAP,Potash (0-0-60)",NaN
429,11/17/2022,Ceres,111,Spread/Spray,VRF MAP: 139 lbs/ac\r\nVRF Potash: 121 lbs/ac,NaN,NaN,NaN,"MAP,Potash (0-0-60)",NaN
430,11/17/2022,Ceres,33,Spread/Spray,VRF MAP: 154 lbs/ac,NaN,NaN,NaN,MAP,NaN
431,11/17/2022,Ceres,41,Spread/Spray,VRF MAP: 115.8 lbs/ac,NaN,NaN,NaN,MAP,NaN


First data class I want to create is event class ass below. As you can see that I created a class with attributes that we don't have data. That is okay, we can leave those attributes optional by adding `blank=True, null=True` (more detail here -> https://docs.djangoproject.com/en/5.1/ref/models/fields/#field-options).

```python 
class event(models.Model):
    # event_date = models.DateTimeField(default=datetime.now, blank=True)
    event_operator = models.CharField(max_length=50)
    event_operation = models.CharField(choices=EVENT_TYPES, max_length=10)
    event_location = models.CharField(max_length=30)

    def __str__(self):
        return self.event_id
```

You can think of a database class as a table. So, let's create an `event` table (dataframe). We will use Pandas' Unique fucntion to see how many event rows we need to create. (more detail about unique function -> https://pandas.pydata.org/docs/reference/api/pandas.unique.html)

In [6]:
# ! check this DK
log_df['operation'].unique() # find a unique value in dataframe's `event` column

array(['Spread/Spray', 'Tillage', 'Plant', 'Harvest', 'Soil Sampled'],
      dtype=object)

The result from unique function is an array which is iterable. We will iterate over the array and create a dataframe. There are several ways to create a dataframe. But I recommend doing by using a list of dict or a dict of list. I will show you the first case. 

In [8]:
# create a list which we will add dictionaries later
event = [] # suffix s is common way to indicate a list (array)

# ! check this DK
for operation in log_df['operation'].unique(): # we iterate over the result from unique function
    event.append( # append new element to the list
        { # curly bracket indicate the starting of dictionary
            "operation": operation, # each key-value pair
        } # the end of dictionary
    )
event # check the outcome

[{'operation': 'Spread/Spray'},
 {'operation': 'Tillage'},
 {'operation': 'Plant'},
 {'operation': 'Harvest'},
 {'operation': 'Soil Sampled'}]

Now we have a list of dictionaries. Making a dataframe is just one function. Note that I leave `lastname` and `phone` emtyp. This is an explicit way to work with optional attribute. You can also just ignore that attribute like I did with `middlename`. You will get the same outcome. 

In [9]:
# create a dataframe
event_df = pd.DataFrame(event)
event_df

,operation
0,Spread/Spray
1,Tillage
2,Plant
3,Harvest
4,Soil Sampled


The next step is to structure the data as required (check data format above). We will need to add `model` which is the class's name in the database. And `pk` or primary key which is the row's index. We want JSON at the end, but JSON is pretty much like dictionary in Python. So, we just need to create a (list of) dictionary.

In [10]:
events = [] # create an empty list
for index, row in event_df.iterrows(): # we iterate over the dataframe row by row
    events.append( # append new element to the list
        { # curly bracket indicate the starting of dictionary
            "model": "acrelog.event",  # the class name
            "pk": index+1, # index starts at 0 while primary key starts at 1. 
            "fields": { # this is a dictionary of attributes
                "operation": row['operation'] # only need firstname for now
            }
        } # the end of dictionary
    )
events # see the result

[{'model': 'acrelog.event', 'pk': 1, 'fields': {'operation': 'Spread/Spray'}},
 {'model': 'acrelog.event', 'pk': 2, 'fields': {'operation': 'Tillage'}},
 {'model': 'acrelog.event', 'pk': 3, 'fields': {'operation': 'Plant'}},
 {'model': 'acrelog.event', 'pk': 4, 'fields': {'operation': 'Harvest'}},
 {'model': 'acrelog.event', 'pk': 5, 'fields': {'operation': 'Soil Sampled'}}]

Now we have the data in the format that we need. To convert into JSON, you can just run the following command. Then you can copy the result and check with https://jsonlint.com/

In [11]:
json.dumps(events)

'[{"model": "acrelog.event", "pk": 1, "fields": {"operation": "Spread/Spray"}}, {"model": "acrelog.event", "pk": 2, "fields": {"operation": "Tillage"}}, {"model": "acrelog.event", "pk": 3, "fields": {"operation": "Plant"}}, {"model": "acrelog.event", "pk": 4, "fields": {"operation": "Harvest"}}, {"model": "acrelog.event", "pk": 5, "fields": {"operation": "Soil Sampled"}}]'

In [13]:
# or write the JSON to the file directly 
with open('operation-data.json', 'w') as fp:
    json.dump(events, fp=fp, indent=4)

You might wonder why we create a dictionary then convert into a dataframe just for turn it into a dictionary again. Can you just make a dictionary that follows the required stucture directly? Absolutely yes. In some cases, that will be the better way to deal with the data. But there is a usecase for a dataframe as well. You will see it soon. For now, let's create another data class. 

```python
class Operation(models.Model):
    date = models.DateField()
    note = models.CharField(max_length=300, blank=True, null=True)
    event = models.ForeignKey(event, on_delete=models.CASCADE)
```
In this class, it has a foreignkey that points to event class. So, instead of recording event name, we will keep the primary key that points to that event. For better understanding, I will trim the original dataframe to keep only the data that we will play with.

In [ ]:
# you can subset the dataframe with any columns (and order)
sub_log_df = log_df[['date', 'event', 'note']]
sub_log_df

We will use the map function (more detail -> https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html) to map from the event's name to primary key. The map functions take several options. But we will use dictionary. The dictionary we want has event's names as keys and primary keys as values. Do you feel that this structure looks familar. Yes, that is the event dataframe that we created earlier. 

In [ ]:
event_df

As I mentioned earlier, dataframe is made from a list of dictionaries or a dictionary of lists. We can convert it back as well.

In [ ]:
event_df.to_dict()

The format you see above is a dictionary of list. We want to use firstname to map with primary key (or dataframe's index). So we will only care about the `firstname`.

In [ ]:
event_df.to_dict()['firstname']

Almost done. We want a dictionary that keys are firstname and values are primary keys. But what we got is opposite. No problem, we just need to inverse it (and add 1 to pandas' index to make it become primary key)

In [ ]:
index_name = event_df.to_dict()['firstname']
name2pk = dict((v, k+1) for k, v in index_name.items())
name2pk

We got it. Now, let map the event's name to primary key.

In [ ]:
sub_log_df['event'] = sub_log_df['event'].map(name2pk) 
sub_log_df

Before we make the final dictionary, we need to deal with date format. Django's SQLite expects `YYYY-MM-DD` format. So, we need to fix our date column a little bit. What you see is only string (a series of charactor). Python has a date (or datetime) variable for dealing with this type of data. Date data type is numerial. So, we can do calulation and also, in our case, format in the way we want. First, we need to convert string to datetime. 

In [ ]:
pd.to_datetime(sub_log_df['date'])

As you can see, we have a date column which we can do many thing on it. But we will just convert it back into string (with the format that we want). Then we will make a dictionary and JSON. The function we use is `strftime` (string from time).

In [ ]:
sub_log_df['date'] = pd.to_datetime(sub_log_df['date']).dt.strftime('%Y-%m-%d')
sub_log_df

Now, we have everything ready. Let's create the final dictionary.

In [ ]:
logs = []
for index, row in sub_log_df.iterrows():
    logs.append({
        "model": "acrelog.operation", 
        "pk": index+1,
        "fields": {
            "date": row['date'],
            "event": row['event'],
            "note": row['note'],
        }
    })
logs

You can dump the dictionary to JSON then load the JSON file to the database.